# Extract irregular time series features from eICU dataset
## We consider those features
* hemoglobin
* creatinine
* sodium
* BUN

## Overview of the procedures
1. Draw $N$ patients randomly from `pivoted_lab`, which is the preprocessed lab measurements provided by official eICU repository `https://github.com/MIT-LCP/eicu-code/blob/master/concepts/pivoted/pivoted-lab.sql`, do this for both train and test splits.
2. Extract above time series features between minute $t_0$ and $T$, we use $t_0 = 0$ and $T = 1440$ in this study (first 24 hours of unit stay)
3. Round time to nearest $r$ minutes, we use $r = 5$ (every 5 minutes), we expand the time dimension for every patient, so that the values over the entire first 24 hours is considered as part of the data, missingness is included. This will introduce a lot of missingness, but this is an inherent nature of sparse, irregular time series data like lab measurements.
4. Add flags indicating whether a value is present or not, then do a cumulative sum on this flag (this is useful for Neural CDE since it indicates the presence of values overtime)

In [1]:
from helpers.loader import DataBaseLoader
loader = DataBaseLoader(user='mt361', password="tian01050417", dbname="eicu_gen", schema="eicu")

In [2]:
import timeit

import joblib
import numpy as np
from tqdm import tqdm

TIMER = timeit.default_timer

PATIENT_DF = loader["patient_id"]

EXTRACT_SQL = """
    SELECT
        patientunitstayid,
        chartoffset,
        hemoglobin,
        creatinine,
        sodium,
        bun
    FROM
        pivoted_lab
    WHERE
        chartoffset >= {start_time} AND chartoffset <= {end_time}
        AND patientunitstayid = {id}
"""

def extract_patient(id, start_time, end_time):
    extracted_df = loader.query(EXTRACT_SQL.format(start_time = start_time, end_time = end_time, id = id))
    if len(extracted_df) == 0:
        return None
    else:
        return extracted_df

def drop_patientunitstayid(df):
    if isinstance(df, type(None)):
        return None
    else:
        return df.drop(columns = ["patientunitstayid"])

def get_all_ids():
    return loader.query("SELECT DISTINCT(patientunitstayid) FROM patient_id").values.flatten()

def get_all_time(start_time, end_time, every_minute=5):
    return np.sort(np.unique((loader.query(f"SELECT DISTINCT(chartoffset) FROM pivoted_lab WHERE chartoffset >= {start_time} AND chartoffset <= {end_time}").values.flatten() // every_minute) * every_minute))

def round_time(df, every_minute=5):
    if isinstance(df, type(None)):
        return None
    else:
        df["chartoffset"] = np.round(df["chartoffset"].values / every_minute) * every_minute
        return df

def expand_time_dim(df, all_times, fill_forward = False):
    if isinstance(df, type(None)):
        return None
    else:
        # Custom aggregation function
        def custom_aggregate(series):
            non_nans = series.dropna()
            if not non_nans.empty:
                return non_nans.median()
            return np.nan
    
        # Group by chartoffset and aggregate
        df = df.groupby('chartoffset').agg(custom_aggregate).reset_index()

        # Reindex with all_times
        df = df.set_index("chartoffset")
        df = df.reindex(all_times, method="ffill" if fill_forward else None)
        df = df.reset_index()
        df = df.rename(columns={"index": "chartoffset"})
        return df
    
def add_nonnan_cum_flag(df):
    if isinstance(df, type(None)):
        return None
    else:
        for col in df.columns:
            if col != "chartoffset":
                df[f"{col}_nonnan"] = np.cumsum(~np.isnan(df[col].values))
        return df

def reverse_to_nonnan_flag(df):
    if isinstance(df, type(None)):
        return None
    else:
        arr = []
        for col in df.columns:
            if "nonnan" in col:
                flag =  np.concatenate([df[col].to_numpy()[:1], df[col].to_numpy()[1:] - df[col].to_numpy()[:-1]])
                arr.append(flag)
        arr = np.stack(arr, axis=1)
        return arr

def add_mortality_flag(df, id):
    mortality = PATIENT_DF[PATIENT_DF["patientunitstayid"] == id]["hospital_expire_flag"].values[0]
    dummy = np.full((df.shape[0], ), mortality)
    df["mortality"] = dummy                             # mortality flag is at the last column
    return df
    
def random_selection(df, num):
    return df.sample(n=num, replace=False, random_state=2023)

def collect_all_patients(all_ids, start_time, end_time, every_minute=5, prefix=""):
    all_times = get_all_time(start_time, end_time, every_minute=5)          # get all rounded times
    all_patients = []
    
    with tqdm(total=len(all_ids), desc=f"Collecting {len(all_ids)} patients...") as pbar:
        
        last_time = TIMER()
        for id in all_ids:
            patient_df = drop_patientunitstayid(extract_patient(id=id, start_time=start_time, end_time=end_time))       # extract patient and drop id
            if patient_df is None:
                continue            # skip if does not exist
            patient_df = round_time(patient_df, every_minute=every_minute)              # round time of charttime to nearest 5 minutes
            patient_df = expand_time_dim(patient_df, all_times)                         # expand time dimension to include all times
            patient_df = add_nonnan_cum_flag(patient_df)                                # add nonnan cumulative flag to indicate missingness
            patient_df = add_mortality_flag(patient_df, id)                             # add mortality flag
            
            nonnan_flag = reverse_to_nonnan_flag(patient_df)                            # reverse process, this is simply for checking, not used as part of the training data
            assert np.all(nonnan_flag == (~patient_df[["hemoglobin", "creatinine", "sodium", "bun"]].isna()).to_numpy()), "FAILED!"
            
            all_patients.append(patient_df.to_numpy())
            pbar.update(1)
            
            if TIMER() - last_time >= 120:
                pbar.set_description(f"Collected {np.asarray(all_patients).shape}... (Saving)")
                last_time = TIMER()
                joblib.dump(all_patients, f"data/eicu-extract/{prefix}irregular_all_patients_{start_time}_{end_time}_{every_minute}.joblib")
            
    joblib.dump(all_patients, f"data/eicu-extract/{prefix}irregular_all_patients_{start_time}_{end_time}_{every_minute}.joblib")
    print("Done!")

In [3]:
from helpers.utils import seed_everything
from sklearn.model_selection import train_test_split
seed_everything(2023)
sub_patient_df = random_selection(PATIENT_DF, 50000)
X, y = sub_patient_df.drop('hospital_expire_flag', axis=1), sub_patient_df['hospital_expire_flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2023, stratify=y, test_size=0.2)
print(f"Mortality rate in train set: {y_train.mean():.3f}, mortality rate in test set: {y_test.mean():.3f}\nNumber of patients in train set: {len(y_train)}, number of patients in test set: {len(y_test)}")

Seed set to 2023

Mortality rate in train set: 0.088, mortality rate in test set: 0.088
Number of patients in train set: 40000, number of patients in test set: 10000


In [4]:
train_ids = X_train['patientunitstayid'].values
test_ids = X_test['patientunitstayid'].values

In [5]:
collect_all_patients(
    all_ids = train_ids, start_time = 0, end_time = 1440, 
    every_minute = 5, prefix="TRAIN_"
)

Collected (36778, 289, 10)... (Saving):  94%|█████████▍| 37527/40000 [1:09:37<04:35,  8.98it/s]  


Done!


In [6]:
collect_all_patients(
    all_ids = test_ids, start_time = 0, end_time = 1440, 
    every_minute = 5, prefix="TEST_"
)

Collected (8963, 289, 10)... (Saving):  94%|█████████▍| 9387/10000 [16:46<01:05,  9.33it/s]


Done!


## Pre-process into tensor and some checking

In [7]:
import joblib
import torch
import numpy as np

test = np.asarray(joblib.load("data/eicu-extract/TEST_irregular_all_patients_0_1440_5.joblib"))
train = np.asarray(joblib.load("data/eicu-extract/TRAIN_irregular_all_patients_0_1440_5.joblib"))
print(test.shape)
print(train.shape)

(8963, 289, 10)
(36778, 289, 10)


In [10]:
test = torch.tensor(test).permute(0, 2, 1)
train = torch.tensor(train).permute(0, 2, 1)
print(test.shape)
print(train.shape)

torch.Size([8963, 10, 289])
torch.Size([36778, 10, 289])


check class imbalance

In [15]:
def imbalance(data_tensor):
    data_tensor = data_tensor[:, 9, :]          # last channel is label
    return data_tensor.sum() / data_tensor.numel()

print(f"Imbalance for test: {imbalance(test)*100:.3f}%")
print(f"Imbalance for train: {imbalance(train)*100:.3f}%")

Imbalance for test: 8.368%
Imbalance for train: 8.554%


save

In [18]:
torch.save(test, "data/eicu-extract/TEST_irregular_all_patients_0_1440_5.pt")
torch.save(train, "data/eicu-extract/TRAIN_irregular_all_patients_0_1440_5.pt")